In [1]:
import pygeohash as gh
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append(os.path.dirname(sys.path[0]))

from python.data_cleaning import prepare_data_for_groupby
from python.grouped_data import data_for_analysis
from python import mlmodel

2022-12-09 10:57:27.156859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 10:57:27.656984: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 10:57:27.657167: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 10:57:29.548721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
df_accidents=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1999-2021.csv', index_col=0)
df_accidents.head()

/tmp/ipykernel_14723/1322138315.py:1: DtypeWarning: Columns (1,3,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft-road-casualty-statistics-accident-1999-2021.csv', index_col=0)


: 

: 

In [ ]:
test=prepare_data_for_groupby(df_accidents,6)
test.head()

In [ ]:
hashes=pd.DataFrame(test[['geohash','accident_year']].groupby('geohash').count())
hashes.columns=['Accidents']
hashes.sort_values('Accidents',inplace=True,ascending=False)
hashes[hashes['Accidents']>=1000].index

Index(['gcpvj0', 'gcpvhc', 'gcpvj4', 'gcpvj1', 'gcpuv2', 'gcpvh3', 'gcpvhf',
       'gcpvh9', 'gcpvjh', 'gcpvnh'],
      dtype='object', name='geohash')

In [ ]:
FOLD_LENGTH=416 #each fold spans over 8 years
FOLD_STRIDE=52 #we have a fold every 1 year
INPUT_LENGTH=10 #every X_i sequence is 10 weeks long
TRAIN_TEST_RATIO=.80 #in each fold, we use 3 years for training and 1 year for test
OUTPUT_LENGTH=10 #we want to predict the number of accidents in the next 10 weeks (y_i length)
SEQUENCE_STRIDE = 1

In [59]:
X_train=[]
y_train=[]
X_test=[]
y_test=[]
for hash in hashes[hashes['Accidents']>=1500].index:
    data=test[test['geohash']==hash]
    data_series=data_for_analysis(data,'W')
    folds=mlmodel.get_folds(data_series,FOLD_LENGTH,FOLD_STRIDE)
    for fold in folds:
        (fold_train, fold_test) = mlmodel.train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)
        X_train_fold, y_train_fold = mlmodel.get_X_y_strides(fold_train, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)
        X_test_fold, y_test_fold = mlmodel.get_X_y_strides(fold_test, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)   
        X_train.append(X_train_fold)
        X_test.append(X_test_fold)
        y_train.append(y_train_fold)
        y_test.append(y_test_fold)
np.array(X_test).shape, np.array(y_train).shape,   
    

((190, 1, 12, 3), (190, 13, 12, 1))

In [60]:
X_train_all_hashes=np.array(X_train[0])
X_test_all_hashes=np.array(X_test[0])
y_train_all_hashes=np.array(y_train[0])
y_test_all_hashes=np.array(y_test[0])

for i in range(1,np.array(X_train).shape[0]):
    X_train_all_hashes=np.vstack((X_train_all_hashes,X_train[i]))
    y_train_all_hashes=np.vstack((y_train_all_hashes,y_train[i]))
    
for i in range(1,np.array(X_test).shape[0]):
    X_test_all_hashes=np.vstack((X_test_all_hashes,X_test[i]))
    y_test_all_hashes=np.vstack((y_test_all_hashes,y_test[i]))


In [1]:
X_train_all_hashes.shape

NameError: name 'X_train_all_hashes' is not defined

In [62]:
   
baseline_model = mlmodel.init_baseline(OUTPUT_LENGTH)
mae_baseline = baseline_model.evaluate(X_test_all_hashes, y_test_all_hashes, verbose=0)[1]
list_of_mae_baseline_model.append(mae_baseline)

from tensorflow.keras.callbacks import EarlyStopping
##### LSTM Model
model = mlmodel.init_model(X_train_all_hashes)
es = EarlyStopping(monitor = "val_mae",
                    mode = "min",
                    patience = 2,
                    restore_best_weights = True)
history = model.fit(X_train_all_hashes, y_train_all_hashes,
                    validation_split = 0.3,
                    shuffle = False,
                    batch_size = 32,
                    epochs = 50,
                    callbacks = [es],
                    verbose = 0)
res = model.evaluate(X_test_all_hashes, y_test_all_hashes, verbose=0)
mae_lstm = res[1]

print(f"MAE LSTM = {round(mae_lstm, 2)}")

##### Comparison LSTM vs Baseline
print(f"Improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")


MAE LSTM = 2.3
Improvement over baseline: 21.02 % 



: 

In [ ]:
X=